In [1]:
%%javascript
requirejs.config({
    paths: {
        'GitgraphJS': ['//cdn.jsdelivr.net/npm/@gitgraph/js?noext'],
    },
});

window.getGitGraph = function (element, callback) {
    require(['GitgraphJS'], function(GitgraphJS) {
        // Get the graph container HTML element.
        var graphContainer2 = element[0];
        console.log(graphContainer2);

        // Instantiate the graph.
        var gitgraph = GitgraphJS.createGitgraph(graphContainer2, {
            orientation: 'horizontal',
            mode: 'compact'
        })
        callback(gitgraph);
    });
}

<IPython.core.display.Javascript object>

# Basic git via command-line

This exercise abuses Jupyter notebooks a little to run bash commands (starting with `!`). We will walk through a relatively simple git scenario, with the intent of giving a hands-on understanding of what's happening under the Github/Sourcetree/etc hood.

In [2]:
!git init

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


This has created the folder `.git` - that's git's entire memory as long as it is on our computer. Most git commands from here on will consult that directory to find out, store and amend git's history.

In [3]:
!git status

On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	sample_data/

nothing added to commit but untracked files present (use "git add" to track)


`git status` is the simplest way to get a snapshot of the current repository as you are using it - what's edited, what's not.

In [4]:
!mkdir -p test

Sneakily, we have already told git not to worry about the `Introduction to git.ipynb` and `example.py` files (more info below).

In [5]:
!git status

On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	sample_data/

nothing added to commit but untracked files present (use "git add" to track)


You will notice that the creation of a `test` directory was not noticed - directories only matter to git if they contain files that it is (or would) track...

In [ ]:
!cp ../example.py test

We have now added a file to the test directory...

In [8]:
!ls test

example.py


In [9]:
!git status

On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	sample_data/
	test/

nothing added to commit but untracked files present (use "git add" to track)


Now you can see `test` has appeared as "Untracked" -- git notices it, but is not yet tracking it, versioning it, or otherwise observing it.

In [10]:
!git add test

We tell git that it matters... then we can see it now wants to version (commit) its contents.

In [11]:
!git status

On branch master

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   test/example.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	sample_data/



We snapshot a version - commit it...

In [12]:
!git config --global user.email "abhinav.tk@flaxandteal.co.uk"
!git config --global user.name "abhinavtk7"

In [13]:
!git commit -m "Initial commit"

[master (root-commit) ecef8c1] Initial commit
 1 file changed, 9 insertions(+)
 create mode 100644 test/example.py


And we now have a git repository like any other - it has a first commit, with a file, and whatever happens to that file, we can now roll back to our committed version.

In [14]:
%%javascript

getGitGraph(element, function (gitgraph) {
    console.log(gitgraph);
    var master = gitgraph.branch("master");
    master.commit("Initial commit");
});

<IPython.core.display.Javascript object>

Above is a simple representation of the current git-tree (these are not live-updating so will only be accurate if you're following the steps closely!).

It has a single dot, indicating a commit, beside a branch-name -- master. This is comparable to, for instance, trunk in SVN, but git is a _decentralized_ version control system, and so you have your own `master` branch locally, independent from the "remote" `master` branch (and any other remote branches)

Lets suppose we wish to update example.py - use [this link](https://jh.ev.openindustry.in/hub/user-redirect/edit/python-course/019-git/test/example.py) or go to the file browser, and open `python-course/019-git/test/example.py`

## Status & Committing

Lets begin by improving the doc-string - the text at the start of the file.

Change:

    This is an example Python file

to:

    Text manipulation utilities

and save. Git can tell this has changed:

In [15]:
!git status

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   test/example.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	sample_data/

no changes added to commit (use "git add" and/or "git commit -a")


We can add and commit these changes in two steps (a combined shortcut is possible, but doing them separately is strongly recommended).

In [16]:
!git add test/example.py
!git commit -m "improve utility file docstring"

[master 5be1bf1] improve utility file docstring
 1 file changed, 1 insertion(+), 1 deletion(-)


These are now recorded. You can continue changing the file, and know that you can find and return to this version.

You'll notice that git gives a handy summary of the scale of changes you just committed -- keep an eye on that, it's often an easy mistake to commit files you did not intend ( _if you haven't pushed your changes to a remote git server_ , you may be able to adjust with `git commit`'s `--amend` flag, but _only_ if you haven't yet sent them anywhere!)

In [19]:
%%javascript

getGitGraph(element, function (gitgraph) {
    console.log(gitgraph);
    var master = gitgraph.branch("master");
    master.commit("Initial commit");
    master.commit("improve utility file docstring");
});

<IPython.core.display.Javascript object>

## The Index & Diff

We notice that the `capitalize` method is using the `.lower` method, woops -- edit that, changing `.lower` to `.upper` and save it now.

In [20]:
!python3 test/example.py

TESTING


Git can tell this has changed...

In [21]:
!git status

On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	sample_data/

nothing added to commit but untracked files present (use "git add" to track)


You should notice that there is a line `modified:   test/example.py` indicating that you have changes that are not committed. This means the repo is "dirty".

Moreover, it is under a section `Changes not staged for commit`. Git has a concept of staging - the _index_ - where you add you changes (additions/modifications/removals) and all of the staged changes are committed at once.

You can stage your changes (git add), unstage them, stage a few different changes, but none of it is permanently recorded.

In [22]:
!git add test/example.py && git status

On branch master
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   test/example.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	sample_data/



This time, we only did the first step - `add`. The `test/example.py` change is now "staged", ready to be committed. However, before doing so, we realise that `capitalize` and `upper` are very different things...

Change the example.py text from `.upper()` to `.capitalize()` and save it.

In [23]:
!git status

On branch master
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   test/example.py

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   test/example.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	sample_data/



Now you can explicitly see that we have a record of our "staged" change (.lower->.upper) under "Changes to be committed", but also our not-yet-staged change (.upper->.capitalize).

Lets see the differences between them. Firstly, how is our local copy of `example.py` different from the staged version?

In [24]:
!git diff

diff --git a/test/example.py b/test/example.py
index f0023bd..4423559 100644
--- a/test/example.py
+++ b/test/example.py
@@ -3,7 +3,7 @@ Text manipulation utilities...
 """
 
 def capitalize(text):
-    return text.upper()
+    return text.capitalize()
 
 if __name__ == "__main__":
     print(capitalize("testing"))


This shows several things - firstly, coloured, the difference between two files. It mentions the index (staging area), which is our default reference copy, and it tells us the mode of the file (644: not executable, readable by anyone, editable by its owner). Now lets see the difference between the index and the last actual commit.

In [25]:
!git diff --staged

diff --git a/test/example.py b/test/example.py
index ecb1d6c..f0023bd 100644
--- a/test/example.py
+++ b/test/example.py
@@ -1,5 +1,5 @@
 """
-Text manipulation utilities
+Text manipulation utilities...
 """
 
 def capitalize(text):


The staged flag tells git we want to compare between our last commit (also known as HEAD) and the index - i.e. what is staged, ready to be committed?

We can also ignore the index, staging area, and say, "what is different between my local file and the last commit"?

In [26]:
!git diff HEAD

diff --git a/test/example.py b/test/example.py
index ecb1d6c..4423559 100644
--- a/test/example.py
+++ b/test/example.py
@@ -1,9 +1,9 @@
 """
-Text manipulation utilities
+Text manipulation utilities...
 """
 
 def capitalize(text):
-    return text.upper()
+    return text.capitalize()
 
 if __name__ == "__main__":
     print(capitalize("testing"))


HEAD is a shortcut name for the last commit made on the currently checked out branch (see below) - we could use any other commit here, or branch, or tag, and git will tell us the different between that and our current local file.

In [27]:
!git diff HEAD~1

diff --git a/test/example.py b/test/example.py
index f5b8601..4423559 100644
--- a/test/example.py
+++ b/test/example.py
@@ -1,9 +1,9 @@
 """
-This is an example Python file
+Text manipulation utilities...
 """
 
 def capitalize(text):
-    return text.upper()
+    return text.capitalize()
 
 if __name__ == "__main__":
     print(capitalize("testing"))


We can even ask git to do some maths with commits - the above asks "what are the differences between my current file, and the commit-before-last"?

While these diff outputs can sometimes be hard to read - code editors often make them a bit simpler to read, using interactive features - they are very powerful. You can take a copy of one, and "apply" it as a "patch" to reproduce a set of changes on top of a known code-base.

If you are working with Github, Gitlab or Bitbucket, you will be familiar with this format in the browser - they also provide integrated tools for code review, and identifying when and where something changed, on top of the diff view.

Lets unstage our changes, as we didn't want to commit the change to "upper" any more...

In [28]:
!git reset

Unstaged changes after reset:
M	test/example.py


You can try the git diffs to see what the current situation looks like - the index is no longer different from the last commit, and your local file contains only the "capitalize" change. You can commit it.

In [29]:
!git add test/example.py
!git commit -m "bugfix: capitalize and upper are different concepts"

[master 7bf6636] bugfix: capitalize and upper are different concepts
 1 file changed, 2 insertions(+), 2 deletions(-)


Your git graph - the representation of your commits - now looks like this.

In [30]:
%%javascript

getGitGraph(element, function (gitgraph) {
    console.log(gitgraph);
    var master = gitgraph.branch("master");
    master.commit("Initial commit");
    master.commit("improve utility file docstring");
    master.commit("bugfix: capitalize and upper are different concepts");
});

<IPython.core.display.Javascript object>

You can examine differences between historic commits using their hashes. Hashes are their permanent, unique names - these long random sequences of letters and numbers, but you can refer to a commit by the first few characters, provided they are sufficient to distinguish it from all the other commits. Conventionally, git (and most git-based tools) show the first 6 characters by default.

In [31]:
!git log --oneline

7bf6636 (HEAD -> master) bugfix: capitalize and upper are different concepts
5be1bf1 improve utility file docstring
ecef8c1 Initial commit


## Branches & Merges

So far our commits have been very, well, linear. One after the next - if development was that easy, we wouldn't need complex tools like git. Most of you will have come across Github branches, and merge/pull requests, but its helpful to see that on the command line to understand the how and why.

Our default branch is called `master`. Beyond that, practice varies, but most common flows will have code ultimately being brought back to master, as the most stable, slowly-moving reference version. However, for developing or collaborating on new features and bug-fixes, changes are generally first created on other branches.

Lets create a new branch.

In [32]:
!git branch feature-reverse-string

This has now created a new branch, at the same point as master. However, we are still sitting on master - any commits we do will go there. We need to "checkout" our branch, to switch to it.

In [33]:
!git checkout feature-reverse-string

Switched to branch 'feature-reverse-string'


It is possible to do those in one movement - `git checkout -b feature-reverse-string`, which will create and checkout a new branch for you.

In [35]:
!git status

On branch feature-reverse-string
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/

nothing added to commit but untracked files present (use "git add" to track)


It is generally a good idea to do `git status` at this point (if not before) to make sure you do not have changes you meant to commit to the previous branch, still hanging about.

Here is a short Python function - it exploits the "step" option in slicing to step backwards, instead of forwards, and create a reversed string (or list, etc.).

In [36]:
def reverse(text):
    return text[::-1]

In [37]:
print(reverse("Testing"))

gnitseT


Add this to `example.py` - the reverse function directly below the capitalize function, and the print-statement at the very end of the file, indented to match the `print(capitalize('testing'))` line. We will go more into testing later in the course, but think of this as a minimal way to check our code!

In [38]:
!python3 test/example.py

Testing
gnitseT


Let's now add and commit that.

In [39]:
!git add test/example.py
!git commit -m "added a utility to reverse a string"

[feature-reverse-string 813cd33] added a utility to reverse a string
 1 file changed, 3 insertions(+)


Our git graph looks like this:

In [40]:
%%javascript

getGitGraph(element, function (gitgraph) {
    console.log(gitgraph);
    var master = gitgraph.branch("master");
    master.commit("Initial commit");
    master.commit("improve utility file docstring");
    master.commit("bugfix: capitalize and upper are different concepts");
    var feature = master.branch("feature-reverse-string");
    feature.commit("added a utility to reverse a string")
});

<IPython.core.display.Javascript object>

We could do a merge back into master at this point, but particularly as you are likely to have seen this in one form or other a number of times before, we will look at what happens in the non-trivial case. Lets suppose an urgent bug-report comes in. A hot-fix needs to be created to address the case where text is a list of characters, rather than a string.

In [41]:
another_text = list("another text")

In [42]:
def capitalize(text):
    if type(text) is not str:
        text = ''.join(text)
    return text.capitalize()

capitalize(another_text)

'Another text'

This will fix our problem (it'll do for this example!). However, the change is urgent, so how do we apply the fix, and commit it without accidentally dragging in our new feature?

This is a key point of branches. Having checked our feature branch is clean - i.e. that we have committed any changes - we switch back to master.

In [56]:
!git checkout master

Switched to branch 'master'


You can check the example.py file to confirm that our new feature has gone.

In [ ]:
!git checkout -b fix-ensure-lists-do-not-break-capitalize

We now swap in our capitalize method to example.py. Let's also add this check at the bottom of the file...

In [46]:
print(capitalize(['a', 'b', ' ', 'c']))

Ab c


We add and commit our change...

In [55]:
!git add test/example.py
!git commit -m "ensure lists do not break capitalization utility"

[fix-ensure-lists-do-not-break-capitalize a6a38ba] ensure lists do not break capitalization utility
 1 file changed, 2 insertions(+)


Our git tree now looks like this:

In [48]:
%%javascript

getGitGraph(element, function (gitgraph) {
    console.log(gitgraph);
    var master = gitgraph.branch("master");
    master.commit("Initial commit");
    master.commit("improve utility file docstring");
    master.commit("bugfix: capitalize and upper are different concepts");
    var feature = master.branch("feature-reverse-string");
    feature.commit("added a utility to reverse a string");
    var hotfix = master.branch("fix-ensure-lists-do-not-break-capitalize");
    hotfix.commit("ensure lists do not break capitalization utility");
});

<IPython.core.display.Javascript object>

Git can also visualize this in ASCII-art:

In [58]:
!git log --all --decorate --oneline --graph

* a6a38ba (fix-ensure-lists-do-not-break-capitalize) ensure lists do not break capitalization utility
| * 5f83da1 (HEAD -> master) ensure lists do not break capitalization utility
|/  
| * 813cd33 (feature-reverse-string) added a utility to reverse a string
|/  
* 7bf6636 bugfix: capitalize and upper are different concepts
* 5be1bf1 improve utility file docstring
* ecef8c1 Initial commit


Now, in reality, we would push this to a remote server, and create a merge/pull request to be reviewed, but for simplicity here, let's assume we can merge it ourselves (or that we have pulled these changes from someone else, and are manually merging it for them).

In [59]:
!git checkout master

M	test/example.py
Already on 'master'


In [60]:
!git merge fix-ensure-lists-do-not-break-capitalize

Your local changes to the following files would be overwritten by merge:
  test/example.py

This has just applied the changes from the hotfix to master.

In [61]:
%%javascript

getGitGraph(element, function (gitgraph) {
    console.log(gitgraph);
    var master = gitgraph.branch("master");
    master.commit("Initial commit");
    master.commit("improve utility file docstring");
    master.commit("bugfix: capitalize and upper are different concepts");
    var feature = master.branch("feature-reverse-string");
    feature.commit("added a utility to reverse a string");
    var hotfix = master.branch("fix-ensure-lists-do-not-break-capitalize");
    hotfix.commit("ensure lists do not break capitalization utility");
    master.merge(hotfix);
});

<IPython.core.display.Javascript object>

Let's now go back to our feature branch and merge it into master too.

In [62]:
!git merge feature-reverse-string

Your local changes to the following files would be overwritten by merge:
  test/example.py

In this case, we may see a Merge conflict -- this means that git was not able to automatically merge the hotfix changes already on master, with the feature branch changes you are now merging. Probably because both added lines in the same place, so it doesn't know how to order them, or if only one should be chosen.

In fact, it has left us up-in-the-air - the file mention (test/example.py) contains "merge annotations", textual notes showing what we need to manually fix, before the merge can finish.

You'll find it looks something like this.

```
"""
This is an example Python file
"""

def capitalize(text):
<<<<<<< HEAD
...

=======
...
>>>>>>> feature-reverse-string
```

You have to manually work out which parts between `<<<<<<< HEAD` and `======` should be kept/deleted, and which parts between `======` and `>>>>>>>> feature-reverse-string` should be kept/deleted. Make 100% sure to delete the merge annotations themselves! When you are confident the script should work as intended, double-check:

In [63]:
!python3 test/example.py

Testing


Finally, complete the merge:

In [64]:
!git add test/example.py
!git commit -m "merged lists hotfix and reversing feature"

[master b795181] merged lists hotfix and reversing feature
 1 file changed, 2 deletions(-)


The git tree now looks like:

In [65]:
%%javascript

getGitGraph(element, function (gitgraph) {
    console.log(gitgraph);
    var master = gitgraph.branch("master");
    master.commit("Initial commit");
    master.commit("improve utility file docstring");
    master.commit("bugfix: capitalize and upper are different concepts");
    var feature = master.branch("feature-reverse-string");
    feature.commit("added a utility to reverse a string");
    var hotfix = master.branch("fix-ensure-lists-do-not-break-capitalize");
    hotfix.commit("ensure lists do not break capitalization utility");
    master.merge(hotfix);
    master.merge(feature);
});

<IPython.core.display.Javascript object>

In [66]:
!git log --all --decorate --oneline --graph

* b795181 (HEAD -> master) merged lists hotfix and reversing feature
* 5f83da1 ensure lists do not break capitalization utility
| * a6a38ba (fix-ensure-lists-do-not-break-capitalize) ensure lists do not break capitalization utility
|/  
| * 813cd33 (feature-reverse-string) added a utility to reverse a string
|/  
* 7bf6636 bugfix: capitalize and upper are different concepts
* 5be1bf1 improve utility file docstring
* ecef8c1 Initial commit


It's good to be able to diagnose and manage git problems from the command line - at some point, Github or your Git user interface will not provide all the tools you need, and some complex issues can only be solved via CLI. `git log` has a number of options for pretty-printing the commit history.

## Magic Files

There are a couple of files that live in your gitroot (the top directory in git), and start with `.git`

Two are particularly important for new users...

In [67]:
!cat .gitignore

cat: .gitignore: No such file or directory


In [ ]:
!cat .gitignore

.ipynb_checkpoints
/example.py
/Introduction to git.ipynb
/.gitignore

This file is checked by git - if a pattern in it matches any given file in the directories below, git ignores it. You can use `*` to identify part of a file, or `/` to state that that filename should only only be ignored if in the root directory of your git.

In particular, the final task will highlight that git is not magical - it is a set of files in the `.git` directory. If you copy that directory, you have copied the entire git state - a good way of taking a quick back-up before trying any experiments is to copy your whole git folder, as (if you correctly copy it!) you can then safely delete the original without losing any git information. Lets wipe the git we created...

In [68]:
!rm -rf .git

### Exercise: Switching Lines

Git graphs can get very complex - this project has largely recreated the Paris Metro in one: https://github.com/vbarbaresi/MetroGit

Can you create the NI Railways map in a git tree?

![Translink route map](https://upload.wikimedia.org/wikipedia/commons/e/ea/NI_Railways_Map.svg)

<small>[SVG Map of NI Railways](https://commons.wikimedia.org/wiki/File:NI_Railways_Map.svg). [RaviC](https://commons.wikimedia.org/wiki/User:RaviC) CC-BY-SA 4.0</small>

Ignore any halts (anything that isn't a circle). Hint: you may find it quickest to start from Lanyon Place as your initial commit.

Rather than having to create content every time you want to commit, you can use `git commit --allow-empty -m "Lurgan"`

In [ ]:
!git init .
!git commit --allow-empty -m "Lanyon Place"